# Stablo odlučivanja
Pripremili: Jovan Petrović, Luka Radovanović i Matija Radičević

Ovim notebook-om krećemo da se bavimo prediktivnom analizom. Glavna razlika između eksploratorne i prediktivne analize je ta što se eksploratorna analiza bavi objašnjavanjem pojava koje su se već desile (uočene iz podataka), dok prediktivna analiza pokušava da predvidi buduće pojave.

U mašinskom učenju, problemi kod kojih je potrebno predvideti neku postojeću ciljnu osobinu, tj label, nazivaju se problemima nadgledanog učenja (eng. supervised learning). U zavisnosti od toga kog je tipa label koji želimo da predvidimo, problemi se razlikuju. U slučaju da želimo da predvidimo kategoriju tj. klasu (label može uzeti ograničen broj vrednosti), onda je to problem klasifikacije.

Klasifikacionim problemima se bave mnogi algoritmi kao što su Decision trees, Naive Bayes, knn, Random Forest, Support Vectore Machine i drugi.

Model stabla odlučivanja (decision tree model) je jako dobar način za poboljšanje donošenja odluka u poslovanju.

# Algoritam

Prilikom izgradnje modela stabla odlučivanja glavna nedoumica je šta treba uzeti za da/ne pitanja koja će klasifikovati naše podatke.

Jedna mogućnost je da sami napravimo procenu, međutim to obično nije najbolji način da kategorizujemo podatke. Mnogo bolja opcija bi bila kada bismo mogli da koristimo neki algoritam mašinskog učenja da odredimo koji je najbolji način da kategorizujemo naše podatke. Upravo to je svrha modela stabla odlučivanja.

Osnovna terminologija kod stabla odlučivanja je sledeća. Root node predstavlja koreni čvor, odnosno vrh stabla. Intermidiates nodes imaju strelice koje pokazuju na i od njih. Za kraj, leaves, odnosno listovi su čvorovi na dnu stabla koji nemaju strelice koje pokazuju od njih.

Da bismo bolje objasnili koncept stabla odlučivanja poslužimo se sledećim primerom. Želimo da predvidimo da li je osoba u braku ili ne, a imamo informacije o 240 osoba. Informacije koje imamo su sledeće: da li je prihod veći ili ne od 50000 dinara, kog su pola i da li su stariji od 30 godina. Recimo da su rezultati našeg ispitivanja sledeći.

![title](img/income.png)

![title](img/pol.png)

![title](img/age.png)

Da bismo odredili koje od stabala bolje određuje rezultat, moramo se upoznati sa konceptom zvanim impurity (kvalitet podele), koji se odnosi na činjenicu da nijedan od listova nije dao rešenje u kome su svi u braku ili ne.

Postoji više načina da se izračuna impurity. Mi ćemo koristiti sckit-learn-ovu implementaciju DecisionTreeClassifer-a koja koristi Gini algoritam po difoltu. Način računanja Gini impuruty-ja je predstavljen na sledećim slikama. 

![title](img/ageleft.png)

![title](img/ageright.png)

![title](img/agewhole.png)

Nakon što smo izračunali Gini impurity možemo odrediti i informacionu dobit (information gain) na sledeći način.

![title](img/igain.png)

Sada se proces ponavlja za prihod i pol. Na kraju biramo onu podelu koja ostvaruje najveću informacionu dobit. U ovom slučaju to je prihod, što ima smisla jer je prihod veći od 50000 dinara u snažnoj korelaciji sa osobama koje su u braku.

![title](img/rez.png)

Kada je završen koreni čvor, proces se ponavlja za sve ostale čvorove u stablu.

![title](img/prihod.png)

Međutim, ne možemo vršiti deljenje u beskonačnost. Drugim rečima, moramo da znamo kada da kažemo stablu da stane. Scikit-Learn implementacija DecisionTreeClassifer-a koristi minimum impurity decrease da bi odlučio da li treba izvršiti podelu nekog čvora. 

![title](img/minimpurity.png)

Pretpostavimo da nakon nekoliko iteracija završimo sa sledećim čvorom na levoj strani stabla.

![title](img/1.png)

![title](img/2.png)

Rezultat je veći nego difoltni treshold koji iznosi 0. Zbog toga, podela ovog čvora će biti izvršena. Da je treshold bio ispod nule, deca ovog čvora bi bila uzeta za listove.

# Kod u pajtonu

Za početak, potrebno je importovati sledeće biblioteke.

In [4]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.tree import export_graphviz
from sklearn.externals.six import StringIO 
from IPython.display import Image 
from pydot import graph_from_dot_data
import pandas as pd
import numpy as np

Koristićemo jedan od najpopularnijih datasetova u oblasti mašinskog učenja, iris dataset.

In [6]:
iris = load_iris()
X = pd.DataFrame(iris.data, columns=iris.feature_names)
y = pd.Categorical.from_codes(iris.target, iris.target_names)

Cilj je da izgradimo stablo odlučivanja kako bismo odredili tip cveta ako znamo njegove dimenzije.

In [7]:
X.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


Iako stablo odlučivanja može da radi sa kategoričkim podacima, mi ćemo ipak izvršiti enkodiranje da bismo kasnije kreirali matricu konfuzije. To možemo uraditi zahvaljujući pandas biblioteci.

In [9]:
y = pd.get_dummies(y)
y

,setosa,versicolor,virginica
0,1,0,0
1,1,0,0
2,1,0,0
3,1,0,0
4,1,0,0
...,...,...,...
145,0,0,1
146,0,0,1
147,0,0,1
148,0,0,1


75% seta uzimamo za trening set, dok jednu četvrtimu ostavljamo za test set.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Sledeći korak je da kreiramo i treniramo instancu DecisionTreeClassifer klase.

In [13]:
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Sada ćemo videti kako naš model funkcioniše sa test podacima.

In [16]:
y_pred = dt.predict(X_test)

Da bismo videli koliko dobro je naše stablo odlučivanja predviđalo koristićemo matricu konfuzije. Brojevi na dijagonali matrice odgovaraju tačnim predikcijama, dok brojevi van dijagonale odgovaraju greškama (true positive i false negative).

In [17]:
species = np.array(y_test).argmax(axis=1)
predictions = np.array(y_pred).argmax(axis=1)
confusion_matrix(species, predictions)

array([[13,  0,  0],
       [ 0, 15,  1],
       [ 0,  0,  9]], dtype=int64)

Kao što možemo videti, naše stablo odlučivanja je odradilo odličan posao jer je tačno predvidelo 37 od 38 biljaka.

# Literatura:
1. [Decision Tree Towards Data Science](https://towardsdatascience.com/decision-tree-in-python-b433ae57fb93)
2. [Open Machine Learning Course Medium](https://medium.com/open-machine-learning-course)
3. [Decision Tree, Medium](https://medium.com/deep-math-machine-learning-ai/chapter-4-decision-trees-algorithms-b93975f7a1f1)